In [1]:
! export KAGGLE_USERNAME="merhan" && export KAGGLE_KEY="de9f258bbdad4348ed345f311c22e02b" && kaggle datasets download --unzip -d jangedoo/utkface-new
! ls


 97% 322M/331M [00:06<00:00, 53.7MB/s]
100% 331M/331M [00:06<00:00, 54.6MB/s]
crop_part1  sample_data  UTKFace  utkface_aligned_cropped


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import os
import cv2
from tqdm import tqdm
from skimage import io
from PIL import Image

In [3]:
file_dir = "/content/UTKFace"
batch_size = 64

Y = []
for image in tqdm(os.listdir(file_dir)):
  img = image.split('_')
  Y.append(float(img[0]))


100%|██████████| 23708/23708 [00:00<00:00, 1044325.22it/s]


In [4]:
 
from torch.utils.data import Dataset, TensorDataset
 
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, root_dir, tensor, transform=None):
        self.root_dir = root_dir
        self.tensor = tensor
        self.transform = transform
 
    def __getitem__(self, index):
        files = os.listdir(self.root_dir)
        img = files[index]
        img_name = os.path.join(self.root_dir, img)
        x = Image.open(img_name)
        
        if self.transform:
            x = self.transform(x)
 
        y = self.tensor[index]
 
        return x, y
 
    def __len__(self):
        return len(self.tensor)

In [5]:
train_transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(10),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485],
                                                    [0.229])])

test_transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485],
                                                    [0.229])])


In [6]:
data = CustomTensorDataset(file_dir, torch.FloatTensor(Y), train_transform)

test = int(len(data)*0.2)  
valid = int(len(data)*0.1)

test_data = torch.utils.data.Subset(data, range(test)) 
test_data.dataset.transform = test_transform

valid_data = torch.utils.data.Subset(data, range(test, test+valid))
valid_data.dataset.transform = test_transform

train_data = torch.utils.data.Subset(data, range(valid + test, len(data)))

train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

In [7]:
Loaders ={
    'train' : train_loader,
    'valid' : valid_loader,
    'test' : test_loader
}

In [8]:
print(len(train_data), len(valid_data), len(test_data))

16597 2370 4741


In [9]:
model_resnet = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

for name, param in model_resnet.named_parameters():
    if("bn" not in name):
        param.requires_grad = False
num_classes = 116

model_resnet.fc = nn.Sequential(nn.Linear(model_resnet.fc.in_features,512), nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))

train_on_gpu = torch.cuda.is_available()

# if GPU is available, move the model to GPU
if train_on_gpu:
    model_resnet = model_resnet.cuda()


Using cache found in /root/.cache/torch/hub/pytorch_vision_master


In [10]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)

In [19]:
def Train(model, n_epochs, optimizer, criterion, Loaders):
  for epoch in range(1, n_epochs+1):

      # keep track of training and validation loss
      train_loss = 0.0
      Train_loss = 0.0
      ###################
      # train the model #
      ###################
      # model by default is set to train
      model.train()
      for batch_i, (data, target) in enumerate(Loaders['train']):
          # move tensors to GPU if CUDA is available
          data = data.repeat(1, 3, 1, 1)
          print(data.size())
          data, target = data.cuda(), target.cuda()
          # clear the gradients of all optimized variables
          optimizer.zero_grad()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          # calculate the batch loss
          
          loss = criterion(output, target.type(torch.cuda.FloatTensor))
          # backward pass: compute gradient of the loss with respect to model parameters
          loss.backward()
          # perform a single optimization step (parameter update)
          optimizer.step()
          # update training loss 
          train_loss += loss.item()
          
          if batch_i % 20 == 19:    # print training loss every specified number of mini-batches
              print('Epoch %d, Batch %d loss: %.16f' %
                    (epoch, batch_i + 1, train_loss / 20))
              Train_loss += train_loss/20
              train_loss = 0.0

          
          #/////////////////////     Validation        \\\\\\\\\\\\\\\\\\\
          
      model.eval() # prep model for evaluation
      for data, target in Loaders['valid']:
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          # calculate the loss
          loss = criterion(output, target)
          # update running validation loss 
          valid_loss += loss.item()*data.size(0)


      # print training loss per epoch
      print('Epoch %d, training loss: %.10f  validation loss : %.10f' %
            (epoch, Train_loss, valid_loss))
      Train_loss = 0.0
      valid_loss = 0.0
  return model

In [ ]:
epochs = 10 # Ex
model_resnet = Train(model_resnet, epochs, optimizer, criterion, Loaders)

In [ ]:
#############################     Testing      ###################################
def Test(model, Loaders, criterion)
  test_loss = 0 
  total = 0
  correct = 0 
  batch =0 
  for data, target in test_loader:
      # move tensors to GPU if CUDA is available
      data = data.repeat(1, 3, 1, 1)
      if train_on_gpu:
          data, target = data.cuda(), target.cuda()
      # forward pass: compute predicted outputs by passing inputs to the model
      output = model(data)
      # calculate the batch loss
      loss = criterion(output, target)
      # update  test loss 
      test_loss += loss.item()
      _, predicted = torch.topk(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      batch +=1
  # calculate avg test loss
  test_loss = test_loss/batch
  
  
  return test_loss, (correct / total)


In [ ]:
loss, accuracy = Test(model_resnet, Loaders, criterion)
print('Test Loss: {:.6f}\n'.format(loss))
print('\nTest Accuracy (Overall): {:.3f}\n'.format(accuracy))